In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# author:XiaoYi
import pickle, os
from Extkinter import *
from PIL import Image,ImageTk
from datetime import datetime
import pandas as pd
from tkinter import *
import tkinter as tk
from tkinter import ttk
import tkinter.messagebox
import time
import numpy as np

In [2]:
def InputBox(default_value, x, y):
    entry = ttk.Entry(window, width=10)
    entry.place(x=x, y=y)
    entry.insert(0, default_value)

def InputBoxChiefComplaintRef(default_value, x, y):
    entry = ttk.Entry(window,width=30)
    entry.place(x=x, y=y)
    entry.insert(0, default_value)
    
def AgeTranslation(birth):
    if birth != 'null' and birth != '':
        if "/" in birth:
            birth_d = datetime.strptime(birth, "%Y/%m/%d")
        elif "-" in birth:
            birth_d = datetime.strptime(birth, "%Y-%m-%d").date()
        else:
            birth_d = datetime.strptime(birth, "%Y%m%d").date()
        today_d = datetime.now()
        age = today_d.year - birth_d.year
        return age
    else:
        return '空值'

def update_patient_info():
    global idx, Name, ID, Gender, Age, Temp, PulseRate, Respiration
    global BloodPres, SpO2, ArrMode, ChiefComp, EndIndex

    if idx < EndIndex:
        Name = data['病人姓名'][idx]
        ID = data['病人ID'][idx]
        Gender = data['性别'][idx]
        Age = AgeTranslation(data['出生日期'][idx])
        Temp = data['T℃'][idx]
        PulseRate = data['P(次/分)'][idx]
        Respiration = data['R(次/分)'][idx]
        BloodPres = data['BP(mmHg)'][idx]
        SpO2 = data['SpO2'][idx]
        ArrMode = data['到院方式'][idx]
        ChiefComp = data['分诊印象'][idx]

        # 更新GUI中的患者信息
        InputBox(Name, 160, 0)
        InputBox(ID, 160, 30)
        InputBox(Gender, 160, 60)
        InputBox(Age, 160, 90)
        InputBox(Temp, 60, 200)
        InputBox(PulseRate, 60, 230)
        InputBox(Respiration, 60, 260)
        InputBox(BloodPres, 60, 290)
        InputBox(SpO2, 60, 320)
        InputBox(ArrMode, 400, 0)
        name_refer.delete(1.0, tk.END)
        name_refer.insert(tk.END, ChiefComp)
        Level = LevelModeLabel.get()
        Department = DepartmentLabel.get()        
    else:
        tk.messagebox.showinfo("提示", "已经到达数据末尾，将关闭窗口。")
        window.destroy()  # 关闭窗口

def creat_csv(file_path, COLUMN):
    if not os.path.exists(file_path):
        tabel = pd.DataFrame(columns=COLUMN)
        tabel.to_csv(file_path, index=False, header=True, encoding="utf_8_sig")
    else:
        tabel = pd.read_csv(file_path, engine='python')
    return tabel

def write(save_path, variables, columns):
    # 如果文件不存在，创建一个新的 CSV 文件
    try:
        DATA = pd.read_csv(save_path)
    except FileNotFoundError:
        DATA = pd.DataFrame(columns=columns)
    new_row = pd.Series(variables, index=columns)
    DATA= pd.concat([DATA, new_row.to_frame().transpose()], ignore_index=True)
    DATA.to_csv(save_path, index=False,encoding="utf_8_sig")

In [3]:
if __name__ == '__main__':
    # 全局变量
    window = None
    data = None
    Name, ID, Gender, Age, Temp, PulseRate, Respiration, BloodPres, SpO2, ArrMode, ChiefComp, Level, Department= [None] * 13
    name_refer = None  # 声明为全局变量
    
    window = tk.Tk()
    window.title('智能分诊系统v1.0-测试版本')
    window.geometry('620x400')
    window.resizable(0, 0)
    
    path = r"./data/ClearData.txt"
    NameDoctor = input('请输入分诊人员的姓名（建议中文）：')
    StartIndex = int(input('请输入起始索引(比如从1开始)：'))
    if StartIndex == 0:
        StartIndex = 1
    EndIndex = int(input('请输入结束索引（比如到100结束）：'))
    ResultPath = r'./result'
    if not os.path.exists(ResultPath):
         os.makedirs(ResultPath)
    save_path = os.path.join(ResultPath, NameDoctor + '_TriageData.csv')
    Columns = ['病人ID','病人姓名','性别','出生日期','到院方式','分诊印象','T℃', 'P(次/分)',
               'R(次/分)', 'BP(mmHg)', 'SpO2', '分诊耗时(秒)', '级别','去向', '级别M','科室M']
    Data = pd.read_csv(path, engine='python')
    P = pd.DataFrame(Data) #, columns=Columns   
    data = P[StartIndex-1: EndIndex]
        
    Tabel = creat_csv(save_path, Columns)
    INDEX = Columns    
    HistoryData = pd.read_csv(save_path, engine='python')
    if   StartIndex >= len(HistoryData):#数据右边
        counter = StartIndex-1
    elif 0<= StartIndex<len(HistoryData) and EndIndex>len(HistoryData):
        counter = len(HistoryData)
    elif EndIndex <=len(HistoryData):#数据之间
#         tk.messagebox.showinfo("提示", "该部分已处理过，将关闭窗口。")
#         window.destroy()
        print('该部分已处理过！！')
        counter = len(HistoryData)
#         os._exit(0)# 括号要放值os._exit(0)--正常退出
    idx = counter
    
    Name = data['病人姓名'][idx]
    ID = data['病人ID'][idx]
    Gender = data['性别'][idx]
    Age = AgeTranslation(data['出生日期'][idx])

    Temp = data['T℃'][idx]
    PulseRate = data['P(次/分)'][idx]
    Respiration = data['R(次/分)'][idx]
    BloodPres = data['BP(mmHg)'][idx]
    SpO2 = data['SpO2'][idx]
    ArrMode = data['到院方式'][idx]
    ChiefComp = data['分诊印象'][idx]
    name_refer = tk.Text(window, width=30, height=3, wrap=tk.WORD)
    name_refer.insert(tk.END, ChiefComp)
    name_refer.place(x=400, y=100)

    # *************************************患者信息***********************
    # *************************************患者信息***********************
    tk.Label(window, text="", bg="lightgray", fg="white").place(x=0, y=0, width=250, height=140)
    tk.Label(window, text="患者信息", bg="lightblue", fg="white", font=("华文细黑", 12)).place(x=15, y=165)

    # 第4步，加载 wellcome image
    image_text = Text(window, height=10, width=16)
    photo = PhotoImage(file="RC.png")
    image_text.image_create('0.0', image=photo)
    image_text.grid(row=2, column=1, columnspan=2, sticky=W)

    # 第5步
    tk.Label(window, text='姓名:', bg="lightgray", font=('华文细黑', 12)).place(x=120, y=0)
    InputBox(Name, 160, 0)
    tk.Label(window, text='ID:', bg="lightgray", font=('华文细黑', 12)).place(x=120, y=30)
    InputBox(ID, 160, 30)
    tk.Label(window, text='性别:', bg="lightgray", font=('华文细黑', 12)).place(x=120, y=60)
    InputBox(Gender, 160, 60)
    # # 单选钮，男
    # def show():
    #     print(Gender[sex.get()-1][0])
    # sex = tk.IntVar(value=1)
    # Gender = [("男", 0),("女", 1),]
    # radioMan = tk.Radiobutton(window, variable=sex, value=1, text='男', bg="lightgray", font=('华文细黑', 12),command=show)
    # radioMan.place(x=160, y=60, width=50, height=20)
    # radioWoman = tk.Radiobutton(window, variable=sex, value=2, text='女', bg="lightgray", font=('华文细黑', 12),command=show)
    # radioWoman.place(x=210, y=60, width=70, height=20)

    tk.Label(window, text='年龄:', bg="lightgray", font=('华文细黑', 12)).place(x=120, y=90)
    InputBox(Age, 160, 90)

    # 第6步，*************************************生命体征
    # 第6步，*************************************生命体征
    tk.Label(window, text="", bg="darkseagreen", fg="black").place(x=0, y=160, width=160, height=200)
    tk.Label(window, text="生命体征", bg="olivedrab", fg="white", font=("华文细黑", 12)).place(x=0, y=165)
    tk.Label(window, text='体温:', bg="darkseagreen", fg="black", font=('华文细黑', 12)).place(x=15, y=200)
    InputBox(Temp, 60, 200)
    tk.Label(window, text='心率:', bg="darkseagreen", fg="black", font=('华文细黑', 12)).place(x=15, y=230)
    InputBox(PulseRate, 60, 230)
    tk.Label(window, text='呼吸:', bg="darkseagreen", fg="black", font=('华文细黑', 12)).place(x=15, y=260)
    InputBox(Respiration, 60, 260)
    tk.Label(window, text='血压:', bg="darkseagreen", fg="black", font=('华文细黑', 12)).place(x=15, y=290)
    InputBox(BloodPres, 60, 290)
    tk.Label(window, text='血氧:', bg="darkseagreen", fg="black", font=('华文细黑', 12)).place(x=15, y=320)
    InputBox(SpO2, 60, 320)

    # *************************************到院方式
    # *************************************到院方式
    ArrivalModeTXT = tk.Label(window, text='到院方式:', font=('华文细黑', 12)).place(x=300, y=0)
    InputBox(ArrMode, 400, 0)

    # ArrivalMode = ['步入', '轮椅', '120', '扶入']
    # ArrivalModeLabel = tk.ttk.Combobox(window, values=tuple(ArrivalMode)).place(x=400, y=0, width=60, height=20)

    tk.Label(window, text='主诉(输入):', font=('华文细黑', 12)).place(x=300, y=40)
    name_input = tk.Text(window, width=30, height=3).place(x=400, y=40)

    tk.Label(window, text='主诉(参考):', font=('华文细黑', 12)).place(x=300, y=100)
    name_refer = tk.Text(window, width=30, height=3, wrap=tk.WORD)
    name_refer.insert(tk.END, ChiefComp)
    name_refer.place(x=400, y=100)


    # InputBoxChiefComplaintRef(ChiefComp, 400, 100)
    #*************************************等级和科室
    #*************************************等级和科室
    LevelTXT = tk.Label(window, text='等级:', font=('华文细黑', 16)).place(x=280, y=200)
    LevelS = ['四级', '三级', '二级','一级' ]
    global LevelModeLabel, DepartmentLabel
    LevelModeLabel = tk.ttk.Combobox(window, values=tuple(LevelS), state="readonly")
    LevelModeLabel.place(x=360, y=200, width=80, height=20)
    
    DepartmentTXT = tk.Label(window, text='科室:', font=('华文细黑', 16)).place(x=450, y=200)
    DepartmentS = ['外科','内科','耳鼻喉', '产科','妇科', '眼科', '神经内科', '神经外科', '骨科', '创伤救治中心']

    DepartmentLabel = tk.ttk.Combobox(window, values=tuple(DepartmentS), state="readonly")
    DepartmentLabel.place(x=510, y=200, width=100, height=20)

    # 第7步，#***************************   计数器
    tk.Label(window, text='已完成:', font=('华文细黑', 10)).place(x=180, y=320)
    label = tk.Label(window, text=len(HistoryData),font=('华文细黑','10'))
    label.place(x=230, y=320)
    
    start = time.time()
    def button_click():
        SingleData=[]
        global counter, idx, Level, Department, Consumption, start, end
        counter += 1
        label.config(text=str(counter))
        idx = counter
        Level = LevelModeLabel.get()
        Department = DepartmentLabel.get()
        end = time.time()
        Consumption = np.round((end - start), 3)
        start = time.time()
              
        SingleData = [ID, Name, Gender, Age, ArrMode, ChiefComp, Temp,
                      PulseRate, Respiration, BloodPres, SpO2, Consumption, str(data['级别'][idx-1]), str(data['去向'][idx-1])]
        SingleData.extend([Level, Department])
        write(save_path, SingleData, INDEX)
        if idx >= EndIndex:
            tk.messagebox.showinfo("提示", "已经到达数据末尾，将关闭窗口。")
            window.destroy()
        else:
            update_patient_info()
    # 第7步，#*************************************确认 和 下一条 按钮
    btn_ok = tk.Button(window, text='OK', font=('Times New Roman', 15), width=10, height=2)
    btn_ok.place(x=280, y=300)
    
    btn_next = tk.Button(window, text='Next', font=('Times New Roman', 15), width=10, height=2, command=button_click)
    btn_next.place(x=450, y=300)
    
    # 第10步，主窗口循环显示
    window.mainloop()

请输入分诊人员的姓名（建议中文）：123
请输入起始索引(比如从1开始)：1
请输入结束索引（比如到100结束）：50


In [11]:
StartIndex,  EndIndex, idx, len(data)

(5, 15, 11, 11)

In [12]:
StartIndex > len(HistoryData)

False

In [13]:
0<= StartIndex<len(HistoryData) and EndIndex>len(HistoryData)

True

In [14]:
EndIndex <=len(HistoryData)

False

In [15]:
data

,病人ID,病人姓名,性别,出生日期,分诊时间,分诊修改时间,分诊印象,分诊耗时,二次分诊,到院方式,T℃,P(次/分),R(次/分),BP(mmHg),SpO2,去向,级别
4,128864987,张祎萱,女,1994/3/8,2022/6/30 12:18,2022/6/30 12:18,乳腺炎,0分26秒,否,步入,36.2,空值,空值,空值,空值,外科,四级
5,3562567,王盼,女,1989/9/1,2022/6/30 12:13,2022/6/30 12:13,输血制品,0分45秒,否,步入,36,空值,空值,空值,空值,内科,四级
6,100840538,王桂伶,女,1953/4/14,2022/6/30 12:03,2022/6/30 12:03,胸闷 心悸,0分35秒,否,步入,空值,107,空值,154/94,97,内科,四级
7,102548773,濮宗芃,男,1934/12/5,2022/6/30 12:02,2022/6/30 12:02,肠梗阻,0分46秒,否,步入,36,空值,空值,空值,空值,外科,四级
8,3957554,严兰花,女,1963/7/26,2022/6/30 12:01,2022/6/30 12:01,胸闷 心慌,0分25秒,否,步入,36,73,空值,142/74,99,内科,四级
9,111143683,韩语珊,女,2008/9/27,2022/6/30 11:59,2022/6/30 11:59,动物致伤,0分25秒,否,步入,36,空值,空值,空值,空值,外科,四级
10,22975429,郝海燕,女,1974/3/19,2022/6/30 11:52,2022/6/30 11:52,动物致伤,0分25秒,否,步入,36,空值,空值,空值,空值,外科,四级
11,3957558,黄昆,男,1960/5/23,2022/6/30 11:37,2022/6/30 11:37,动物致伤,2分19秒,否,步入,36.2,空值,空值,空值,空值,外科,四级
12,103557159,文昕,男,1951/7/12,2022/6/30 11:34,2022/6/30 11:34,眼睑肿胀,0分40秒,否,步入,36.2,空值,空值,空值,空值,眼科,四级
13,105837373,李娜,女,1983/1/9,2022/6/30 11:32,2022/6/30 11:32,动物致伤,0分56秒,否,步入,36.2,空值,空值,空值,空值,外科,四级


In [29]:
NAME = HistoryData[-1:]['病人姓名'].values[0]
INDEX_Final = data['病人姓名'].str.contains(NAME).index[0]
if len([INDEX_Final]) == 1:
    FinalIndex = INDEX_Final
else: 
    print(INDEX_Final)
    FinalIndex = INDEX_Final[-1]

In [30]:
FinalIndex

3

In [7]:
HistoryData = pd.read_csv(save_path, engine='python')
NAME = HistoryData[-1:]['病人姓名'].values[0]
NAME

'骆嘉丽'

In [15]:
duplicates = data['病人姓名'].duplicated(keep=False)
data['病人姓名'][duplicates].tail(1).index[0]

6

6    骆嘉丽
Name: 病人姓名, dtype: object

In [14]:
import pandas as pd

# 创建一个示例 DataFrame
data0 = {'Column1': [1, 2, 3, 1, 4, 2, 5, 6, 2]}
df = pd.DataFrame(data0)

# 列中的重复值
duplicates = df['Column1'].duplicated(keep='first')

# 获取最后一个重复值的行
first_occurrence_indices = []
for value in df['Column1'][duplicates]:
    print(value)
    first_occurrence_indices.append(df[df['Column1'] == value].index[0])

1
2
2


In [15]:
first_occurrence_indices

[0, 1, 1]

In [17]:
df['Column1'][duplicates].index[0]

Int64Index([3, 5, 8], dtype='int64')

In [18]:
df['Column1'][duplicates]

3    1
5    2
8    2
Name: Column1, dtype: int64

In [4]:
import pandas as pd
HistoryData = pd.read_csv(save_path, engine='python')
len(HistoryData)

# 假设你的数据为 HistoryData
# 添加一个列表示级别预测是否正确
HistoryData['级别预测正确'] = HistoryData['级别'] == HistoryData['级别M']

# 添加一个列表示去向预测是否正确
HistoryData['去向预测正确'] = HistoryData['去向'] == HistoryData['科室M']

# 添加一个列表示同时级别和去向都正确
HistoryData['同时正确'] = HistoryData['级别预测正确'] & HistoryData['去向预测正确']

# 计算同时正确的比例
级别正确率 = HistoryData['级别预测正确'].mean()
科室正确率 = HistoryData['去向预测正确'].mean()
同时正确比例 = HistoryData['同时正确'].mean()

print(f'级别预测正确率: {100 * 级别正确率:.2f}%')
print(f'科室预测正确率: {100 * 科室正确率:.2f}%')
print(f'同时级别和去向都正确的比例: {100 * 同时正确比例:.2f}%')

级别预测正确率: nan%
科室预测正确率: nan%
同时级别和去向都正确的比例: nan%
